In [ ]:
%pip -q install -e ../
%pip show iesta

In [ ]:

import iesta.machine_learning.dataloader
#from .autonotebook import tqdm as notebook_tqdm
import iesta.stats.significance
import matplotlib.pyplot as plt

%matplotlib inline
import seaborn as sns

sns.reset_defaults()
sns.set(
    rc={'figure.figsize':(3,2)}, 
    #style="white" # nicer layout
)


In [ ]:
_, feature_dfs = iesta.machine_learning.dataloader.load_training_features_df()

In [ ]:
significance = iesta.stats.significance.run_all_significance_test(feature_dfs)

In [ ]:
for k in significance.keys():
    print(k)


def _get_key(
    ideology, disregard_okay=False, undersampled=False, binary_effect=False
):
    _okay = "_Okay" if disregard_okay else ""
    _undersampled = "_undersampled" if undersampled else ""
    _binary = "_binary" if binary_effect else ""
    _key = f"{ideology}{_binary}_effect{_undersampled}{_okay}_all_features"
    return _key


def _get_significance(
    ideology, disregard_okay=False, undersampled=False, binary_effect=False
):
    _key = _get_key(ideology, disregard_okay, undersampled, binary_effect)
    _df = significance[_key][0].reset_index().copy()
    _df = _df[
        (~_df["feature"].str.endswith("_count"))
        & (~_df["feature"].str.endswith("_ratio"))
    ]
    if "Unnamed: 0" in _df.columns.tolist():
        del _df["Unnamed: 0"]
    return _df

In [ ]:
# LIBERAL BINARY - With NO OKAY

liberal_binary_nookay = _get_significance(
    "liberal", disregard_okay=True, undersampled=False, binary_effect=True
)
liberal_binary_nookay_undersampled = _get_significance(
    "liberal", disregard_okay=True, undersampled=True, binary_effect=True
)
liberal_binary_nookay, liberal_binary_nookay_undersampled

In [ ]:
cons_binary_nookay = _get_significance(
    "conservative", disregard_okay=True, undersampled=False, binary_effect=True
)
cons_binary_nookay_undersampled = _get_significance(
    "conservative", disregard_okay=True, undersampled=True, binary_effect=True
)
cons_binary_nookay, cons_binary_nookay_undersampled

In [ ]:
liberal_alleffect_nookay = _get_significance(
    "liberal", disregard_okay=True, undersampled=False, binary_effect=False
)
liberal_alleffect_nookay_undersampled = _get_significance(
    "liberal", disregard_okay=True, undersampled=True, binary_effect=False
)
liberal_alleffect_nookay
# liberal_alleffect_nookay_undersampled HAS 1 RESULT

In [ ]:
conservative_alleffect_nookay = _get_significance(
    "conservative",
    disregard_okay=True,
    undersampled=False,
    binary_effect=False,
)
conservative_alleffect_nookay_undersampled = _get_significance(
    "conservative", disregard_okay=True, undersampled=True, binary_effect=False
)
conservative_alleffect_nookay
# liberal_alleffect_nookay_undersampled HAS 1 RESULT

In [ ]:
_get_key(
    "liberal", disregard_okay=True, undersampled=False, binary_effect=False
)

In [ ]:
features_df = feature_dfs[1]["liberal"]

features_df["effect"].unique()

In [ ]:
features_df[~features_df["effect"].isin(["okay"])].copy()["effect"].unique()